# Label the service statuses in a uniform way

In [26]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np

In [28]:
root_dir = '/home/shane/Documents/thesis/output/parsed/final/'
files = glob.glob(root_dir + '*.csv')
for file in files:
    df = pd.read_csv(file)
    print(file.split('/')[8], len(df))

cloud-amazon-web-services.csv 291
slack.csv 1835
outage-report.csv 120018
downdetector_v2.csv 323708
downrightnow.csv 11904
atlassian.csv 23864


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


github-status.csv 9612
cloudflare-status.csv 8433
discord.csv 1836
cloud-google-cloud-platform.csv 22811


In [35]:
unique_statuses = set()

for file in files:
    if 'outage' in file:
        continue
        
    df = pd.read_csv(file)
    statuses = df.iloc[:, 4:]
    for s in statuses:
        for t in df[s].unique():
            unique_statuses.add(str(t))

### The unique status types we should encode

In [36]:
unique_statuses

{' operational',
 ' partial_outage',
 '0',
 '1',
 '2',
 '3',
 '?',
 'All services available',
 'Cloud Developer Tools reporting issues',
 'Cloud Machine Learning reporting issues',
 'Cloud Run reporting issues',
 'Cloud Spanner reporting issues',
 'Degraded Performance',
 'Google App Engine reporting issues',
 'Google BigQuery reporting issues',
 'Google Cloud Composer reporting issues',
 'Google Cloud Console reporting issues',
 'Google Cloud Dataflow reporting issues',
 'Google Cloud Dataproc reporting issues',
 'Google Cloud Datastore reporting issues',
 'Google Cloud Functions reporting issues',
 'Google Cloud Networking reporting issues',
 'Google Cloud Pub/Sub reporting issues',
 'Google Cloud SQL reporting issues',
 'Google Cloud Storage reporting issues',
 'Google Cloud Support reporting issues',
 'Google Cloud infrastructure components reporting issues',
 'Google Compute Engine reporting issues',
 'Google Kubernetes Engine reporting issues',
 'Google Stackdriver reporting issu

#### Encode in a dict, as follows:
##### 0 -> operational; 1 -> partial outage/issues; 2 -> full outage/major problems

In [42]:
labels_dict = {
   ' operational': 0,
   ' partial_outage': 1,
   '0': '',
   '1': '',
   '2': '',
   '3': '',
   '?': '?',
   'All services available': '',
   'Cloud Developer Tools reporting issues': '',
   'Cloud Machine Learning reporting issues': '',
   'Cloud Run reporting issues': '',
   'Cloud Spanner reporting issues': '',
   'Degraded Performance': '',
   'Google App Engine reporting issues': '',
   'Google BigQuery reporting issues': '',
   'Google Cloud Composer reporting issues': '',
   'Google Cloud Console reporting issues': '',
   'Google Cloud Dataflow reporting issues': '',
   'Google Cloud Dataproc reporting issues': '',
   'Google Cloud Datastore reporting issues': '',
   'Google Cloud Functions reporting issues': '',
   'Google Cloud Networking reporting issues': '',
   'Google Cloud Pub/Sub reporting issues': '',
   'Google Cloud SQL reporting issues': '',
   'Google Cloud Storage reporting issues': '',
   'Google Cloud Support reporting issues': '',
   'Google Cloud infrastructure components reporting issues': '',
   'Google Compute Engine reporting issues': '',
   'Google Kubernetes Engine reporting issues': '',
   'Google Stackdriver reporting issues': '',
   'Identity & Security reporting issues': '',
   'Likely Service Disruption': '',
   'Major Outage': '',
   'Multiple services reporting issues': '',
   'No issues': '',
   'Operational': '',
   'Operations reporting issues': '',
   'Partial Outage': '',
   'Possible Service Trouble': '',
   'Recent Signs of Service Trouble': '',
   "Something's not quite right View details": '',
   'Under Maintenance': '',
   'nan': '',
   'Up': '',
   'alert-danger': '',
   'alert-success': '',
   'alert-warning': '',
   'degraded_performance': '',
   'major_outage': '',
   'operational': '',
   'partial_outage': '',
   'under_maintenance': ''
}